# Project 15. Modelo valor de mercado coches

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

### Inicialización

In [1]:
import math
import numpy as np
import pandas as pd 

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from matplotlib import pyplot as plt

### Carga de datos

In [2]:
data = pd.read_csv('datasets/P15/car_data.csv')

### Información general y visualización inicial de datos

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [5]:
data.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
241141,30/03/2016 23:52,7600,wagon,2006,manual,150,3er,150000,1,gasoline,bmw,no,30/03/2016 00:00,0,50765,06/04/2016 00:17
52729,28/03/2016 16:50,6500,small,2010,manual,69,ibiza,90000,2,petrol,seat,no,28/03/2016 00:00,0,91174,06/04/2016 22:17
316771,22/03/2016 12:36,4500,sedan,2005,manual,82,a_klasse,150000,2,gasoline,mercedes_benz,yes,22/03/2016 00:00,0,38304,06/04/2016 02:16
347853,08/03/2016 10:51,2500,small,2004,manual,61,picanto,150000,6,petrol,kia,no,08/03/2016 00:00,0,83278,20/03/2016 19:48
249688,17/03/2016 14:49,9999,wagon,2009,manual,105,golf,150000,12,gasoline,volkswagen,no,17/03/2016 00:00,0,48308,06/04/2016 21:17


### Corrección y preparación de datos
En el paso anterior encontramos valores nulos presentes en diferentes columnas, sin embargo nuestro target:'Price' está completo, veamos como manejar estos datos. Así mismo, se observaron valores de 0 para la columna 'Power' lo cual es inconsistente y podría ser un error por tanto debemos hacer cambios en esta información. Verificaremos los valores de la columna 'NumberOfPictures' ya que tanto en la vista de los primeros 5 resultados y en la muestra aleatoria todos los resultados fueron de 0. Finalmente verificaremos que tan relevantes son algunas características para el target.

In [6]:
#Ya que nuestra columna target está completa reemplazaremos los valores nulos de las columnas:
#VehicleType, Gearbox, Model, FuelType y NotRepaired, por 'unknown' para no perder información
null_info_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']

data[null_info_columns] = data[null_info_columns].fillna('unknown')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        354369 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            354369 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              354369 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           354369 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        354369 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [7]:
#Veamos como están distribuidos los valores de la columna 'Power'
data['Power'].describe()

count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

In [8]:
print(data['Power'].value_counts())

Power
0        40225
75       24023
60       15897
150      14590
101      13298
         ...  
1433         1
11025        1
1992         1
10910        1
15017        1
Name: count, Length: 712, dtype: int64


Vemos que hay valores en power que no tienen ningún sentido, valores de 0 o valores mayor a 1000 son practicamente imposibles y claramente afectarían nuestro modelo, debemos eliminar estos valores.

In [9]:
data = data[data['Power'] <= 1000]
data = data[data['Power'] > 0]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313842 entries, 1 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        313842 non-null  object
 1   Price              313842 non-null  int64 
 2   VehicleType        313842 non-null  object
 3   RegistrationYear   313842 non-null  int64 
 4   Gearbox            313842 non-null  object
 5   Power              313842 non-null  int64 
 6   Model              313842 non-null  object
 7   Mileage            313842 non-null  int64 
 8   RegistrationMonth  313842 non-null  int64 
 9   FuelType           313842 non-null  object
 10  Brand              313842 non-null  object
 11  NotRepaired        313842 non-null  object
 12  DateCreated        313842 non-null  object
 13  NumberOfPictures   313842 non-null  int64 
 14  PostalCode         313842 non-null  int64 
 15  LastSeen           313842 non-null  object
dtypes: int64(7), object(9)
me

Ahora que hemos hecho las correciones en la característica 'Power', veamos los valores de la columna 'NumberOfPictures':

In [10]:
print(data['NumberOfPictures'].value_counts())

NumberOfPictures
0    313842
Name: count, dtype: int64


In [11]:
#Ya que todos los valores de esta columna son 0, no aportará nada a nuestros modelos, por tanto la eliminaremos completamente
data =  data.drop(columns='NumberOfPictures', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313842 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        313842 non-null  object
 1   Price              313842 non-null  int64 
 2   VehicleType        313842 non-null  object
 3   RegistrationYear   313842 non-null  int64 
 4   Gearbox            313842 non-null  object
 5   Power              313842 non-null  int64 
 6   Model              313842 non-null  object
 7   Mileage            313842 non-null  int64 
 8   RegistrationMonth  313842 non-null  int64 
 9   FuelType           313842 non-null  object
 10  Brand              313842 non-null  object
 11  NotRepaired        313842 non-null  object
 12  DateCreated        313842 non-null  object
 13  PostalCode         313842 non-null  int64 
 14  LastSeen           313842 non-null  object
dtypes: int64(6), object(9)
memory usage: 38.3+ MB


Las características 'DateCrawled', 'DateCreated', 'PostalCode' y 'LastSeen' realmente no aportan mucha información directamente a nuestro target asi que pasaremos a eliminarlas tambien.

In [12]:
data = data.drop(columns=['DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313842 entries, 1 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              313842 non-null  int64 
 1   VehicleType        313842 non-null  object
 2   RegistrationYear   313842 non-null  int64 
 3   Gearbox            313842 non-null  object
 4   Power              313842 non-null  int64 
 5   Model              313842 non-null  object
 6   Mileage            313842 non-null  int64 
 7   RegistrationMonth  313842 non-null  int64 
 8   FuelType           313842 non-null  object
 9   Brand              313842 non-null  object
 10  NotRepaired        313842 non-null  object
dtypes: int64(5), object(6)
memory usage: 28.7+ MB


Veamos los valores de las demás características numéricas

In [13]:
#Empecemos por 'RegistrationYear'
print(data['RegistrationYear'].value_counts())
data['RegistrationYear'].describe()


RegistrationYear
1999    20149
2000    19401
2005    19200
2006    18725
2001    18258
        ...  
1933        1
8200        1
1947        1
1944        1
3500        1
Name: count, Length: 111, dtype: int64


count    313842.000000
mean       2003.483775
std          35.336413
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

In [14]:
#Claramente el valor mínimo de 1000 o máximo de 9999 no tienen sentido debemos eliminar los valores que no tienen sentido
data = data[data['RegistrationYear'] < 2025]
data = data[data['RegistrationYear'] > 1900]
data['RegistrationYear'].describe()

count    313805.000000
mean       2003.247240
std           7.007376
min        1910.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        2019.000000
Name: RegistrationYear, dtype: float64

In [15]:
#verifiquemos la misma información para la columna 'RegistrationMonth'data['RegistrationMonth'].value_counts())
print(data['RegistrationMonth'].value_counts())
data['RegistrationMonth'].describe()

RegistrationMonth
3     31796
6     29030
4     27050
5     26896
7     25025
10    24019
12    22331
11    22068
0     21975
9     21908
1     21185
8     20755
2     19767
Name: count, dtype: int64


count    313805.000000
mean          5.934462
std           3.615522
min           0.000000
25%           3.000000
50%           6.000000
75%           9.000000
max          12.000000
Name: RegistrationMonth, dtype: float64

En el caso del mes, no eliminaremos aquellos donde el mes es 0 ya que lo interpretaremos como desconocido, sin embargo esta categoría la interpretaremos como categórica y reemplazaremos el mes '0' por 'unknown'.

In [16]:
data['RegistrationMonth'] = data['RegistrationMonth'].replace(0, 'unknown').astype(str)
print(data['RegistrationMonth'].value_counts())

RegistrationMonth
3          31796
6          29030
4          27050
5          26896
7          25025
10         24019
12         22331
11         22068
unknown    21975
9          21908
1          21185
8          20755
2          19767
Name: count, dtype: int64


In [17]:
#Veamos la ultima característica numérica
data['Mileage'].describe()

count    313805.000000
mean     128505.520945
std       36805.430101
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Mileage, dtype: float64

Esta ultima columna presenta datos consistentes por tanto no haremos modificaciones en la misma.

In [18]:
#Veamos la distribucion de los valores de nuestro target
data['Price'].describe()

count    313805.000000
mean       4705.696209
std        4590.966346
min           0.000000
25%        1250.000000
50%        2999.000000
75%        6890.000000
max       20000.000000
Name: Price, dtype: float64

In [19]:
#Los valores de 'price' iguales a 0 realmente no nos ayuda para nuestro modelo, las eliminaremos
data = data[data['Price'] > 100]
data['Price'].describe()

count    305064.000000
mean       4840.219590
std        4585.977958
min         101.000000
25%        1350.000000
50%        3199.000000
75%        6990.000000
max       20000.000000
Name: Price, dtype: float64

## Entrenamiento del modelo 

Probaremos inicialmente 3 modelos: arbol de decisión, bosque aleatorio y regresión lineal. para esto utilizaremos 2 pipelines, una para los modelos basados en arbol donde codificaremos con OneHot las características categóricas unicamente, y el segundo para el modelo de regresión lineal donde se codificaran las características categóricas y escalaran las características numéricas con StandardScaler():

In [20]:
#Separemos las características categóricas de las numéricas
num_features = ['RegistrationYear', 'Power', 'Mileage']
cat_features = ['RegistrationMonth', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [21]:
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
cat_transformer =  Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [22]:
tree_preprocessor = ColumnTransformer(
    transformers=[('num', 'passthrough', num_features),
                 ('cat', cat_transformer, cat_features)])

linear_preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, num_features),
                 ('cat', cat_transformer, cat_features)])

In [23]:
tree_pipeline = Pipeline(steps=[
    ('tree_preprocessor', tree_preprocessor),
    ('model', DecisionTreeRegressor(random_state=12345, max_depth=5))])

rand_forest_pipeline = Pipeline(steps=[
    ('tree_preprocessor', tree_preprocessor),
    ('model', RandomForestRegressor(random_state=12345, n_estimators=100, max_depth=5))])

linreg_pipeline = Pipeline(steps=[
    ('linear_preprocessor', linear_preprocessor),
    ('model', LinearRegression())])

In [24]:
#división targets y features
features = data[num_features + cat_features]
target = data['Price']

features.info()
target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305064 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   RegistrationYear   305064 non-null  int64 
 1   Power              305064 non-null  int64 
 2   Mileage            305064 non-null  int64 
 3   RegistrationMonth  305064 non-null  object
 4   VehicleType        305064 non-null  object
 5   Gearbox            305064 non-null  object
 6   Model              305064 non-null  object
 7   FuelType           305064 non-null  object
 8   Brand              305064 non-null  object
 9   NotRepaired        305064 non-null  object
dtypes: int64(3), object(7)
memory usage: 25.6+ MB
<class 'pandas.core.series.Series'>
Index: 305064 entries, 1 to 354368
Series name: Price
Non-Null Count   Dtype
--------------   -----
305064 non-null  int64
dtypes: int64(1)
memory usage: 4.7 MB


In [25]:
#División en conjuntos de train y test
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=12345)

**Árbol de decisión**

In [26]:
%%time
#entrenemos el modelo de árbol
tree_pipeline.fit(train_features, train_target)
test_prediction_tree = tree_pipeline.predict(test_features)
print(root_mean_squared_error(test_target, test_prediction_tree))

2482.9491538755597
CPU times: total: 1.7 s
Wall time: 1.71 s


**Bosque Aleatorio**

In [27]:
%%time
rand_forest_pipeline.fit(train_features, train_target)
test_prediction_forest = rand_forest_pipeline.predict(test_features)
print(root_mean_squared_error(test_target, test_prediction_forest))

2424.759498616539
CPU times: total: 1min 8s
Wall time: 1min 9s


**Regresión Lineal**

In [28]:
%%time
linreg_pipeline.fit(train_features, train_target)
test_prediction_linreg = linreg_pipeline.predict(test_features)
print(root_mean_squared_error(test_target, test_prediction_linreg))

2638.2941291521847
CPU times: total: 30.9 s
Wall time: 4.55 s


**Prueba de cordura**:

In [29]:
baseline_prediction = pd.Series(train_target.mean(), index=test_target.index)
print(root_mean_squared_error(test_target, baseline_prediction))

4593.386468573785


Nuestros resultados indican claramente que nuestros modelos predicen, sin embargo el modelo con el mekor resultado es el de Bosque aleatorio, probemos realizar una validación cruzada con este modelo:

In [30]:
%%time
cv_scores = cross_val_score(rand_forest_pipeline,
                            features,
                            target,
                            cv=5,
                            scoring='neg_root_mean_squared_error')

rmse_scores = -cv_scores
print('RMSE medio:', rmse_scores.mean())

RMSE medio: 2434.524915211746
CPU times: total: 6min 11s
Wall time: 6min 15s


### Potenciación de gradiente:
**LightGBM**

In [34]:
%%time
lgbm_train_features = train_features.copy()
lgbm_test_features = test_features.copy()

for col in cat_features:
    lgbm_train_features[col] = lgbm_train_features[col].astype('category')
    lgbm_test_features[col] = lgbm_test_features[col].astype('category')

model = LGBMRegressor(n_estimators=500, random_state=12345)
model.fit(lgbm_train_features, train_target, categorical_feature=cat_features)
lgbm_test_predictions = model.predict(lgbm_test_features)

print('RMSE:', root_mean_squared_error(test_target, lgbm_test_predictions))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 667
[LightGBM] [Info] Number of data points in the train set: 228798, number of used features: 10
[LightGBM] [Info] Start training from score 4838.317970
RMSE: 1561.4011710459604
CPU times: total: 30.9 s
Wall time: 2.58 s


**CatBoost**

CatBoost tampoco requiere de codificación de características categóricas:

In [33]:
%%time
model = CatBoostRegressor(iterations=500, depth=6, random_seed=12345)
model.fit(train_features, train_target, cat_features=cat_features, verbose=100)
cat_boost_test_predictions = model.predict(test_features)

print('RMSE:', root_mean_squared_error(test_target, cat_boost_test_predictions))

Learning rate set to 0.169711
0:	learn: 4085.7046490	total: 270ms	remaining: 2m 14s
100:	learn: 1703.8259382	total: 15.5s	remaining: 1m 1s
200:	learn: 1631.8602975	total: 30.3s	remaining: 45s
300:	learn: 1597.9023772	total: 44.9s	remaining: 29.7s
400:	learn: 1573.5770983	total: 60s	remaining: 14.8s
499:	learn: 1555.2969716	total: 1m 14s	remaining: 0us
RMSE: 1623.7574911450254
CPU times: total: 12min 45s
Wall time: 1min 16s


## Análisis del modelo

Despues de entrenar y probar los diferen modelos hemos obtenido que en cuanto a nuestros 3 modelos iniciales, el que presentó mejores resultados fue el modelo de Bosque aleatorio con un RMSE: 2424 que aumento ligeramente al realizar una prueba de validación cruzada ademas el timpo de ejecución de dicha celda fue de 1min 9s y el de la validación cruzada fue 6min 15s, claramente un aumento sustancial. Para este modelo se realizó una codificación de características categróricas utilizando OneHotEncoder.

Sin embargo los mejores resultados fueron obtenidos con nuestros modelos de potenciación de gradiente obteniendo un RMSE: 1623 y un tiempo de 1min 16s para el modelo que utilizó CatBoost, una mejora bastante alta en comparación con el modelo de Bosque aleatorio. Y finalmente nuestro mejor modelo, el que utilizó LightGBM, donde obtuvimos un RMSE: 1561 con un tiempo de ejecución de 2.21 s, extremadamente veloz y con la mejor métrica, es claramente muy superior a los demás, y el recomendado con los resultados de este análisis.

Por supuesto, es importante mencionar que aunque evidentemente, como pudimos constatar con nuestra prueba de cordura para el modelo de regresión lineal, nuestros modelos predicen, los resultados no son los mejores y el error es bastante alto para una plataforma que busca valores apropiados para los vehiculos que pondrán a la venta los usuarios. Nuestro modelo claramente requiere mejoras para llegar a tener los resultados que requiere la empresa.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados
- [x]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos